## 파이토치 기본


### 파이토치 공식 튜토리얼
- https://tutorials.pytorch.kr/

### 파이토치 패키지(API)
- `torch` : 메인 네임스페이스. 텐서(기본단위) 등 수학함수 포함. Numpy와 유사한 구조
- `torch.autograd` : 자동미분을 위한 함수들이 포함. 컨텍스트 매니저, 기반클래스 포함
- `torch.nn` / `torch.nn.functional` : neuralnetwork. 신경만 구축위한 데이터 구조, 레이어 등이 정의되어 있는 모듈. ReLU 등도 포함
- `torch.optim` : SGD(확률적 경사 하강법) 중심 파라미터 최적화 알고리즘 포함
- `torch.utils` : SGD 반복 연산 시 배치용 유틸리티 함수 포함
- torch.multiprocessing : 파이토치용 병렬프로세싱 환경을 안전하게 다루기 위한 모듈

### 텐서
- 파이토치 기본단위
    - 1차원 배열 - 벡터
    - 2차원 배열 - 행렬(매트릭스)
    - 3차원 배열 - 텐서

- <img src="../image/ml018.png" width=700>



- <img src="../image/ml019.png" width=700>


In [2]:
import torch
import numpy as np

In [3]:
t1 = torch.tensor([1.0, 2.0, 3.0])
t1

tensor([1., 2., 3.])

In [ ]:
# 텐서의 크기, 텐서 자료형, 자료형 + 전체타입, 어느 디바이스에서 사용중인지
print(t1.shape, t1.dtype, t1.type(), t1.device)

torch.Size([3]) torch.float32 torch.FloatTensor cpu


In [4]:
n1 = np.array([1.0, 2.0, 3.0])
n1

array([1., 2., 3.])

In [18]:
print(n1.shape, n1.dtype)

(3,) float64


In [6]:
## numpy에서 1, 0으로 초기화되는 배
n2 = np.ones([2, 3])
n2

array([[1., 1., 1.],
       [1., 1., 1.]])

In [8]:
n3 = np.zeros([2, 3])
n3

array([[0., 0., 0.],
       [0., 0., 0.]])

In [9]:
t2 = torch.ones(2,3)
t2

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [20]:
print(t2.shape, t2.dtype, t2.type(), t2.device)

torch.Size([2, 3]) torch.float32 torch.FloatTensor cpu


In [11]:
t3 = torch.zeros(2, 3)
t3

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [12]:
# numpy로 만든 배열을 텐서로 변환
t4 = torch.from_numpy(n3)
t4

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

- 넘파이와 파이토피 텐서간의 형변환 아주 쉽다

In [14]:
t5 = torch.ones(3, 3, 3)
t5

tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])

In [21]:
print(t5.shape, t5.dtype, t5.type(), t5.device)
print(t5.size())

torch.Size([3, 3, 3]) torch.float32 torch.FloatTensor cpu
torch.Size([3, 3, 3])


In [33]:
# 랜덤값, randn()은
t6 = torch.rand(2,3)
t6

tensor([[0.1389, 0.9997, 0.3048],
        [0.6412, 0.6299, 0.6964]])

In [22]:
# cpu에 있는 텐서를 cuda로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
# t5(cpu)에 있는 텐서를 cuda로 이동 변수명을 다르게 하면 복사, 변수명을 똑같이 하면 이동
t5 = t5.to(device=device)

In [26]:
print(t5.shape, t5.dtype, t5.type(), t5.device)

torch.Size([3, 3, 3]) torch.float32 torch.cuda.FloatTensor cuda:0


- cuda(GPU 사용하는 것):0(그래픽카드가 다수일 수 있음)
- 파이토치의 텐서 사용 == 넘파이 배열 사용

#### GPU 사용법

In [29]:
# CPU 사용
import torch.nn

# 신경망 모델 생성 - 3개의 입력을 넣어서 하나의 출력을 내는 신경망
model = torch.nn.Linear(3, 1)

sample_input = torch.tensor([[1.0 , 2.0, 3.0]])
output = model(sample_input)

output

tensor([[-0.6317]], grad_fn=<AddmmBackward0>)

In [30]:
# CUDA 사용
import torch.nn

# 신경망 모델 생성 - 3개의 입력을 넣어서 하나의 출력을 내는 신경망
model = torch.nn.Linear(3, 1)
model.to(device)

sample_input = torch.tensor([[1.0 , 2.0, 3.0]]).to(device)
output = model(sample_input)

output

tensor([[0.7640]], device='cuda:0', grad_fn=<AddmmBackward0>)

### PyTorch 모델 학습
- torch.nn.Module로 신경망 모델 만들기 - 텐서플로우(함수)와 차이점 확인
- 손실 함수, 옵티마이저 설정 학습
- 학습 루프 기본 구조 학습

#### 기본 구조 추가

In [ ]:
# 파이토치 모듈 로드
import torch
import torch.nn as nn 
import torch.nn.functional as F # 보통 F로 사용

# 모델 선언
class SimpleNet(nn.Module):         # nn.Module 클래스를 상속해서 클래스 생성
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Linear : Dense Layer(케라스의 Dense()와 동일)
        self.fc1 = nn.Linear(4, 16)     # input : 4개 입력, output 16개 뉴런
        self.fc2 = nn.Linear(16, 3)     # input : 16개 입력, output 3개 클래스(3가지 분류 예측)

    def forward(self, x):           # 실제 연산이 수행되는 함수(자동 호출)
        x = F.relu(self.fc1(x))     # 첫번째 레이어 통과 - 활성화함수(시그모이드, 소프트맥스, 렐루 모두 존재)
        x = self.fc2(x)             # 두번째 레이어 통과 - 출력층(필요시 softmax함수 사용)
        return x 

- 텐서플로우(케라스)는 중간층에서 입력값을 설정하지 않아도 됨
- 파이토치는 입력값, 출력값을 모두 설정해야 함

#### 모델 생성

In [36]:
# 모델 확인
model = SimpleNet()
model

SimpleNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=3, bias=True)
)

In [38]:
# 샘플입력값
x = torch.rand(1, 4)
x

tensor([[0.1130, 0.9247, 0.8398, 0.3765]])

In [39]:
out = model(x)
out

tensor([[ 0.1019, -0.0627, -0.2557]], grad_fn=<AddmmBackward0>)

#### 손실함수/옵티마이저 설정

In [41]:
# 분류(마지막 출력 3개) CrossEntropyLoss - 
criterion = nn.CrossEntropyLoss()

# 옵티마이저(Adam 권장)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)   # lr(learning rate)학습률 : 보통 0.01 ~ 0.0001 사이로 지정

#### 1회 Epoch 학습

In [47]:
y = torch.tensor([1])


# Forward
# 예측
outputs = model(x)
# 손실계산
loss = criterion(outputs, y)

# Backward(역전파) : 뉴런을 통과한 값이 다시 다음 훈련에 입력으로 사용
# 기울기 초기화
optimizer.zero_grad()   # 이전 단계에서 계산된 gradiant(기울기/가중치) 초기화
# 역전파 계산
loss.backward()     # 손실 기준으로 각 파라미터에 대한 gradiant 계산(오차 역전파)
# 파라미터 업데이트
optimizer.step()    # 계산한 gradient를 이용, 파라미터 업데이트(학습훈련)

# 현재 손실값을 출력
loss.item()

0.5659849643707275

### 데이터셋, 데이터로더
- `Dataset`, `DataLoader` 학습
- 내장 데이터셋
- 커스텀 데이터셋 만들기
- 배치 학습 처리

#### 데이터셋, 데이터로더란?
- Dataset - 데이터를 불러오는 방법
- DataLoader - 데이터를 배치 단위로 나누기, 셔플, 병렬처리 방법

#### 추가
- 머신러닝, 딥러닝에 사용되는 데이터샘플 종류 예제

In [50]:
from sklearn.datasets import load_breast_cancer, load_wine  # 11개 데이터셋
import seaborn as sns 

datas = sns.load_dataset('titanic')    # https://github.com/mwaskom/seaborn-data 확인

from torchvision.datasets import FashionMNIST, MNIST, CIFAR10       # 이미지 데이터셋
from keras.datasets import mnist, fashion_mnist, boston_housing, cifar10, cifar100, imdb        # 이미지, 텍스트 데이터셋

In [51]:
datas

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#### 붓꽃(Iris) 데이터셋 사용

In [72]:
# 데이터셋 처리 모듈 로드
from sklearn.datasets import load_iris, load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

In [54]:
# 데이터 로드
iris = load_iris()

## 입력(특성)데이터 X, 타겟 y == input, target
X = iris['data']
y = iris['target']

print(X.shape, y.shape)     # numpy 데이터

(150, 4) (150,)


In [56]:
# sepal(꽃받침) length(cm), sepal width(cm), petal(꽃잎) length(cm), petal width(cm)
X[0, : ]

array([5.1, 3.5, 1.4, 0.2])

In [58]:
# 0(setosa), 1(versicolor), 2(virginica)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [60]:
# 표준화(정규화)
scaler = StandardScaler()   # 사이킷런의 표준정규화 클래스 사용
X = scaler.fit_transform(X)

In [62]:
X[0,:]

array([-0.90068117,  1.01900435, -1.34022653, -1.3154443 ])

In [66]:
# 훈련/테스트세트 분리
train_scaled, test_scaled, train_target, test_target = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
# 훈련/검증세트 분리
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [70]:
print(train_scaled.shape, val_scaled.shape, test_scaled.shape)

(76, 4) (20, 4) (30, 4)


In [71]:
train_target

array([2, 0, 0, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2,
       0, 0, 1, 1, 1, 1, 2, 0, 1, 2, 1, 0, 2, 0, 2, 2, 2, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 2, 1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 1, 1, 2, 1, 1,
       2, 1, 1, 2, 0, 1, 1, 1, 0, 0])

#### 커스텀 데이터셋 만들기

In [73]:
class irisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)   # 특성(입력)값은 실수
        self.y = torch.tensor(y, dtype=torch.long)      # CrossEntropyLoss에서는 long 타입 필요

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [ ]:
# 커스텀 데이터셋으로 생성 - 입력(특성)과 타겟을 하나로 묶음
train_dataset = irisDataset(train_scaled, train_target)
val_dataset = irisDataset(val_scaled, val_target)

In [81]:
# 데이터로더로 생성 - 배치학습, 셔플, 병렬데이터 로딩
trian_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

- 실제 모델 훈련에서는 `DataLoader`를 사용